In [ ]:
!pip install node2vec
!pip install --upgrade gensim
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

  Created wheel for node2vec: filename=node2vec-0.4.3-py3-none-any.whl size=5980 sha256=c23807891dd9832c4e1239e48d46c509c5617f90cfcb3fa274a0b7cf76417651
  Stored in directory: /root/.cache/pip/wheels/07/62/78/5202cb8c03cbf1593b48a8a442fca8ceec2a8c80e22318bae9
Successfully built node2vec
     |████████████████████████████████| 24.1 MB 1.4 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 7.9 MB 13.5 MB/s 
     |████████████████████████████████| 3.5 MB 14.8 MB/s 


In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import pandas as pd
import numpy as np 
import random
random.seed(2022)
np.random.seed(2022)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving data.csv to data.csv
Saving synthetic_edge_index.csv to synthetic_edge_index.csv


In [ ]:
uploaded2 = files.upload()

Saving synthetic_deepwalk_10d.model to synthetic_deepwalk_10d.model
Saving synthetic_Node2Vec_10d.model to synthetic_Node2Vec_10d.model


In [ ]:
uploaded3 = files.upload()

Saving Synthetic_feats_ate.xlsx to Synthetic_feats_ate.xlsx


In [ ]:
import io
df=pd.read_csv(io.BytesIO(uploaded['data.csv']),index_col=0)

from sklearn.model_selection import train_test_split
df_train,df_test = train_test_split(df,test_size = 0.2,random_state=42)
df

,X0,X1,X2,X3,X4,T,y,tau,b,e
0,0.003154,0.402130,0.582037,0.539144,0.414101,0,0.743337,0.202642,0.763640,0.100000
1,0.594773,0.034123,0.250783,0.094229,0.148447,0,0.598364,0.314448,0.356388,0.100000
2,0.403102,0.952464,0.887029,0.141932,0.376224,1,3.017079,0.677783,1.563889,0.900000
3,0.177169,0.367340,0.704904,0.043881,0.904914,0,0.342602,0.272255,0.783347,0.203037
4,0.492973,0.030507,0.002244,0.535752,0.489047,0,-0.501458,0.261740,1.323027,0.100000
...,...,...,...,...,...,...,...,...,...,...
9995,0.989169,0.379821,0.512016,0.444698,0.915355,1,3.888099,0.684495,1.827391,0.900000
9996,0.604783,0.351680,0.938821,0.560349,0.205232,1,1.879708,0.478232,1.667655,0.619563
9997,0.300281,0.457595,0.810301,0.170581,0.925457,0,0.616104,0.378938,1.244277,0.418394
9998,0.513388,0.990300,0.266999,0.770421,0.307555,1,2.053641,0.751844,2.032429,0.900000


In [ ]:
col=df.columns
y_train=torch.from_numpy(np.array((df_train['y']))).reshape(df_train.shape[0],1).to(torch.float32)
y_test=torch.from_numpy(np.array(df_test['y'])).reshape(df_test.shape[0],1).to(torch.float32)
x_train=torch.from_numpy(np.array(df_train[col[:6]])).to(torch.float32)
x_test=torch.from_numpy(np.array(df_test[col[:6]])).to(torch.float32)

train_reverse=df_train[col[:6]].copy()
train_reverse["T"]=1-train_reverse["T"]
train_reverse=torch.from_numpy(np.array(train_reverse)).to(torch.float32)
test_reverse=df_test[col[:6]].copy()
test_reverse["T"]=1-test_reverse["T"]
test_reverse=torch.from_numpy(np.array(test_reverse)).to(torch.float32)

In [ ]:
from scipy.stats import entropy
def R2(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    R2 = 1 - np.sum(np.square(y_actual-y_predicted)) / np.sum(np.square(y_actual-np.mean(y_actual)))
    return R2

def MAE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    MAE = np.mean(abs(y_actual-y_predicted))
    return MAE

def RMSE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    RMSE = np.sqrt(np.mean(np.square(y_actual-y_predicted)))
    return RMSE

def CVRMSE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    RMSE = np.sqrt(np.mean(np.square(y_actual-y_predicted)))
    CVRMSE = RMSE/np.mean(y_actual)
    return CVRMSE

def MAPE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    MAPE=np.mean(abs((y_actual-y_predicted)/y_actual))
    return MAPE

def MSE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    MSE = np.mean(abs(y_actual-y_predicted)**2)
    return MSE

def kl_divergence(y_predicted, y_actual):

    stacked_values = np.hstack((y_predicted, y_actual))
    stacked_low = np.percentile(stacked_values, 0.1)
    stacked_high = np.percentile(stacked_values, 99.9)
    bins = np.linspace(stacked_low, stacked_high, 100)

    distr = np.histogram(y_predicted, bins=bins)[0]
    distr = np.clip(distr / distr.sum(), 0.001, 0.999)
    true_distr = np.histogram( y_actual, bins=bins)[0]
    true_distr = np.clip(true_distr / true_distr.sum(), 0.001, 0.999)

    kl = entropy(distr, true_distr)
    return kl 

In [ ]:
edge_index=pd.read_csv(io.BytesIO(uploaded['synthetic_edge_index.csv']))
edge_index=torch.from_numpy(np.transpose(np.array(edge_index)))

In [ ]:
from gensim.models import word2vec
model_dw=word2vec.Word2Vec.load("synthetic_deepwalk_10d.model")
model_n2v=word2vec.Word2Vec.load("synthetic_Node2Vec_10d.model")

lst_dw=[]
lst_n2v=[]
for i in range(6):
    lst_dw.append(model_dw.wv[i])
    lst_n2v.append(model_n2v.wv[i])
len(lst_n2v)

6

In [ ]:
ate_list=[]
ate=pd.read_excel(io.BytesIO(uploaded3["Synthetic_feats_ate.xlsx"]),index_col=0)
for i in col[:6]:
    ate_list.append(float(ate[ate['Feature']==i]["ATE"].values))
 
ate_list

[0.6138629231035895,
 0.6256793856791922,
 0.001699585840328169,
 0.9901459828003378,
 0.4672027205639851,
 0.5306717299934571]

In [ ]:
import torch
from torch_geometric.data import InMemoryDataset, download_url

# create Mytrain Data
class Mytrain(InMemoryDataset):
    def __init__(self, root,transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
       # ,self.edge_index,self.y=x,edge_index,y
        

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list=[]
        weighted_feats=[]
        for i in range(x_train.shape[0]):
            Edge_index = edge_index.type(torch.long)
            X =x_train[i]
            t=torch.zeros(6,11) # 11 dimensions = 1+10 = node number, node embedding
            
            for j in range(6): # for j-th feature
              t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # current node (1d)
                                                     list(lst_dw[int(j)]), # node embedding (10d), prior embeddings from DeepWalk/Node2Vec     
                                                    ))) 
              
            Y = y_train[i].reshape(-1,1).to(torch.float32)
            data = Data(x=t, edge_index=Edge_index, y=Y)
            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

# create Mytest Data
class Mytest(InMemoryDataset):
    def __init__(self, root,transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
       # ,self.edge_index,self.y=x,edge_index,y
        
   
    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list=[]
        weighted_feats=[]
        for i in range(x_test.shape[0]):
            Edge_index = edge_index.type(torch.long)
            X =x_test[i]
            t=torch.zeros(6,11) # 11 dimensions = 1+10 = node number, node embedding
            
            for j in range(6): # for j-th feature
              t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # current node (1d)
                                                     list(lst_dw[int(j)]), # node embedding (10d), prior embeddings from DeepWalk/Node2Vec     
                                                    ))) 

            Y = y_test[i].reshape(-1,1).to(torch.float32)
            data = Data(x=t, edge_index=Edge_index, y=Y)
            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

# create Mytrain_reverse Data
class Mytrain_reverse(InMemoryDataset):
    def __init__(self, root,transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
       # ,self.edge_index,self.y=x,edge_index,y
        
    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]
    
    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list=[]
        weighted_feats=[]
        for i in range(train_reverse.shape[0]):
            Edge_index = edge_index.type(torch.long)
            X =train_reverse[i]
            t=torch.zeros(6,11) # 11 dimensions = 1+10 = node number, node embedding
            
            for j in range(6): # for j-th feature
              t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # current node (1d)
                                                     list(lst_dw[int(j)]), # node embedding (10d), prior embeddings from DeepWalk/Node2Vec     
                                                    ))) 

            Y = y_train[i].reshape(-1,1).to(torch.float32)
            data = Data(x=t, edge_index=Edge_index, y=Y)
            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])


# create Mytest_reverse Data
class Mytest_reverse(InMemoryDataset):
    def __init__(self, root,transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
       # ,self.edge_index,self.y=x,edge_index,y
        

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list=[]
        weighted_feats=[]
        for i in range(test_reverse.shape[0]):
            Edge_index = edge_index.type(torch.long)
            X =test_reverse[i]
            t=torch.zeros(6,11) # 11 dimensions = 1+10 = node number, node embedding
            
            for j in range(6): # for j-th feature
              t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # current node (1d)
                                                     list(lst_dw[int(j)]), # node embedding (10d), prior embeddings from DeepWalk/Node2Vec     
                                                    ))) 

            Y = y_test[i].reshape(-1,1).to(torch.float32)
            data = Data(x=t, edge_index=Edge_index, y=Y)

            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [ ]:
Mydata_train=Mytrain(".\mydata\MYdata_train")
Mydata_test=Mytest(".\mydata\MYdata_test")
Mytrain_reverse=Mytrain_reverse(".\mydata\MYtrain_reverse")
Mytest_reverse=Mytest_reverse(".\mydata\MYtest_reverse")

Processing...
Done!
Processing...
Done!
Processing...
Done!
Processing...
Done!


# GCN

In [ ]:
import torch
from torch_scatter import scatter_add
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(11, 64)
        self.conv2 = GCNConv(64, 21)
        self.f1 = torch.nn.Linear(126,32)
        self.f2 = torch.nn.Linear(32,1)
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = x.reshape(-1,126)
        x = self.f1(x)
        x = self.f2(x)
        return x

In [ ]:
def train():
    model.train()
    loss_all = 0
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in train_loader:
        loss = 0
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
        y_actual +=(label).cpu().detach().ravel().tolist()
        y_predicted +=(output).cpu().detach().ravel().tolist()
    
    
    loss=loss_all/len(Mydata_train)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2,mse,kl


def val():
    model.eval()
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in test_loader:
      loss = 0
      data = data.to(device)
      output = model(data)
      label = data.y.to(device)
      y_actual +=(label).cpu().detach().ravel().tolist()
      y_predicted +=(output).cpu().detach().ravel().tolist()
      loss = crit(output, label)
      loss_all += loss.item()

    
    loss = loss_all / len(Mydata_test)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2, mse,kl

    

te_train = {}
te_valid = {}
for i in range(5):
    train_mse=[]
    train_kl=[]
    train_error=[]

    valid_mse=[]
    valid_kl=[]
    valid_error=[]

    num_epochs = 2560
    batch_size = 512
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Net().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=5e-4)
    crit = F.mse_loss
    train_loader = DataLoader(Mydata_train, batch_size=batch_size)
    test_loader = DataLoader(Mydata_test, batch_size=512)
    reverse_train = DataLoader(Mytrain_reverse, batch_size=512)
    reverse_test = DataLoader(Mytest_reverse, batch_size=512)
    for epoch in range(num_epochs):
        loss,r2,mse,kl=train()
        if epoch %64==0:      
            print('train_loss:')
            print(loss)

            loss,r2,mse,kl=val()
            print('test_loss:')
            print(loss)
  
    y_predicted = []
    for data in train_loader:
        loss = 0
        data = data.to(device)
        output = model(data)
        label = data.y.to(device)
        y_predicted +=(output).cpu().detach().ravel().tolist()
    df_train["y_hat"]=y_predicted

    y_reversed=[]
    for data in reverse_train:
        data = data.to(device)
        output = model(data)
        y_reversed +=(output).cpu().detach().ravel().tolist()

    df_train["y_reversed"]=y_reversed
    t0=df_train[df_train["T"]==0]
    t0=t0.copy()
    t0.loc[:,"tau_hat"]=t0.loc[:,"y_reversed"]-t0.loc[:,"y_hat"]

    t1=df_train[df_train["T"]==1]
    t1=t1.copy()
    t1.loc[:,"tau_hat"]=t1.loc[:,"y_hat"]-t1.loc[:,"y_reversed"]

    ite_pred=np.concatenate([t1["tau_hat"],t0["tau_hat"]],axis=0)
    ite_true=np.concatenate([t1["tau"],t0["tau"]],axis=0)

    mse=MSE(ite_pred,ite_true)
    kl=kl_divergence(ite_pred,ite_true)
    ate_error=abs((np.mean(ite_pred)/np.mean(ite_true))-1)
    train_mse.append(mse)
    train_kl.append(kl)
    train_error.append(ate_error)

    print("Model %d has MSE on trainset:%f" % (i,np.mean(train_mse)),end='  ')
    print("Model %d has KL on trainset:%f" % (i,np.mean(train_kl)),end='  ')
    print("Model %d has ATE error on trainset:%f" % (i,np.mean(train_error)),end='  ')

    te_train["GCN ATE % Error",i]=np.mean(train_error)
    te_train["GCN MSE",i]=np.mean(train_mse)
    te_train["GCN KL Divergence",i]=np.mean(train_kl)

    y_predicted = []
    for data in test_loader:
        loss = 0
        data = data.to(device)
        output = model(data)
        label = data.y.to(device)
        y_predicted +=(output).cpu().detach().ravel().tolist()
    df_test["y_hat"]=y_predicted

    y_reversed=[]
    for data in reverse_test:
        data = data.to(device)
        output = model(data)
        y_reversed +=(output).cpu().detach().ravel().tolist()

    df_test["y_reversed"]=y_reversed
    t0=df_test[df_test["T"]==0]
    t0=t0.copy()
    t0.loc[:,"tau_hat"]=t0.loc[:,"y_reversed"]-t0.loc[:,"y_hat"]

    t1=df_test[df_test["T"]==1]
    t1=t1.copy()
    t1.loc[:,"tau_hat"]=t1.loc[:,"y_hat"]-t1.loc[:,"y_reversed"]

    ite_pred=np.concatenate([t1["tau_hat"],t0["tau_hat"]],axis=0)
    ite_true=np.concatenate([t1["tau"],t0["tau"]],axis=0)

    mse=MSE(ite_pred,ite_true)
    kl=kl_divergence(ite_pred,ite_true)
    ate_error=abs((np.mean(ite_pred)/np.mean(ite_true))-1)
    valid_mse.append(mse)
    valid_kl.append(kl)
    valid_error.append(ate_error)

    print("Model %d has MSE on validset:%f" % (i,np.mean(valid_mse)),end='  ')
    print("Model %d has KL on validset:%f" % (i,np.mean(valid_kl)),end='  ')
    print("Model %d has ATE error on validset:%f" % (i,np.mean(valid_error)),end='  ')

    te_valid["GCN ATE % Error",i]=np.mean(valid_error)
    te_valid["GCN MSE",i]=np.mean(valid_mse)
    te_valid["GCN KL Divergence",i]=np.mean(valid_kl)


R2:-0.707896  MSE:2.379082  KL:0.509878  MAE:1.246652  RMSE:1.542427  CVRMSE:1.018692  train_loss:
2.3790818510055542
R2:-0.119306  MSE:1.635576  KL:1.773338  MAE:1.023140  RMSE:1.278896  CVRMSE:0.844401  test_loss:
0.0032694076895713807
R2:-0.024616  MSE:1.427280  KL:0.635162  MAE:0.954733  RMSE:1.194688  CVRMSE:0.789029  R2:0.074352  MSE:1.289418  KL:0.848484  MAE:0.910612  RMSE:1.135525  CVRMSE:0.749955  R2:0.104163  MSE:1.247891  KL:0.725248  MAE:0.890023  RMSE:1.117091  CVRMSE:0.737780  R2:0.142331  MSE:1.194724  KL:0.533811  MAE:0.872995  RMSE:1.093034  CVRMSE:0.721892  R2:0.174976  MSE:1.149249  KL:0.380852  MAE:0.853509  RMSE:1.072030  CVRMSE:0.708020  R2:0.184635  MSE:1.135795  KL:0.320053  MAE:0.847078  RMSE:1.065737  CVRMSE:0.703863  R2:0.186710  MSE:1.132905  KL:0.327121  MAE:0.844853  RMSE:1.064380  CVRMSE:0.702967  R2:0.193836  MSE:1.122979  KL:0.323482  MAE:0.842174  RMSE:1.059707  CVRMSE:0.699881  R2:0.190908  MSE:1.127058  KL:0.315869  MAE:0.843413  RMSE:1.061630  CVRM

In [ ]:
te_train,te_valid

({('GCN ATE % Error', 0): 0.2442573234673373,
  ('GCN ATE % Error', 1): 0.2542773073706823,
  ('GCN ATE % Error', 2): 0.26949893023406335,
  ('GCN ATE % Error', 3): 0.28933613378115797,
  ('GCN ATE % Error', 4): 0.28993473798843694,
  ('GCN KL Divergence', 0): 0.19910949767671815,
  ('GCN KL Divergence', 1): 0.2270470242554361,
  ('GCN KL Divergence', 2): 0.2153324713811344,
  ('GCN KL Divergence', 3): 0.251813244004785,
  ('GCN KL Divergence', 4): 0.2588734833131375,
  ('GCN MSE', 0): 0.06692486887023488,
  ('GCN MSE', 1): 0.06601832436268354,
  ('GCN MSE', 2): 0.06789043509449283,
  ('GCN MSE', 3): 0.07484856262157202,
  ('GCN MSE', 4): 0.08000056688659522},
 {('GCN ATE % Error', 0): 0.36933461275800683,
  ('GCN ATE % Error', 1): 0.38399371987533626,
  ('GCN ATE % Error', 2): 0.3918219033738539,
  ('GCN ATE % Error', 3): 0.4262304326834814,
  ('GCN ATE % Error', 4): 0.4346249467267178,
  ('GCN KL Divergence', 0): 0.4105731610732837,
  ('GCN KL Divergence', 1): 0.46649939766305787,
  

# GAT

In [ ]:
from uuid import RFC_4122
import torch
import math
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import GATConv

from torch_geometric.utils import add_self_loops,degree
from torch_geometric.datasets import Planetoid
import ssl
import torch.nn.functional as F


class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.gat1=GATConv(in_channels=11,out_channels=8,heads=8,dropout=0.6)
        self.gat2=GATConv(in_channels=64,out_channels=21,heads=1,dropout=0.6)
        self.f1 = torch.nn.Linear(126,32)
        self.f2 = torch.nn.Linear(32,1)
    def forward(self,data):
        x,edge_index=data.x, data.edge_index
        x=self.gat1(x,edge_index)
        x=self.gat2(x,edge_index)
        x=x.reshape(-1,126)
        x = self.f1(x)
        x = self.f2(x)
        return x

ssl._create_default_https_context = ssl._create_unverified_context

def train():
    model.train()
    loss_all = 0
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in train_loader:
        loss = 0
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
        y_actual +=(label).cpu().detach().ravel().tolist()
        y_predicted +=(output).cpu().detach().ravel().tolist()
    
    
    loss=loss_all/len(Mydata_train)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2,mse,kl


def val():
    model.eval()
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in test_loader:
      loss = 0
      data = data.to(device)
      output = model(data)
      label = data.y.to(device)
      y_actual +=(label).cpu().detach().ravel().tolist()
      y_predicted +=(output).cpu().detach().ravel().tolist()
      loss = crit(output, label)
      loss_all += loss.item()

    
    loss = loss_all / len(Mydata_test)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2, mse,kl

    
for i in range(5):
    train_mse=[]
    train_kl=[]
    train_error=[]

    valid_mse=[]
    valid_kl=[]
    valid_error=[]

    num_epochs = 2560
    batch_size = 512
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Net().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=5e-4)
    crit = F.mse_loss
    train_loader = DataLoader(Mydata_train, batch_size=batch_size)
    test_loader = DataLoader(Mydata_test, batch_size=512)
    reverse_train = DataLoader(Mytrain_reverse, batch_size=512)
    reverse_test = DataLoader(Mytest_reverse, batch_size=512)
    for epoch in range(num_epochs):
        loss,r2,mse,kl=train()
        if epoch %64==0:      
            print('train_loss:')
            print(loss)

            loss,r2,mse,kl=val()
            print('test_loss:')
            print(loss)
  
    y_predicted = []
    for data in train_loader:
        loss = 0
        data = data.to(device)
        output = model(data)
        label = data.y.to(device)
        y_predicted +=(output).cpu().detach().ravel().tolist()
    df_train["y_hat"]=y_predicted

    y_reversed=[]
    for data in reverse_train:
        data = data.to(device)
        output = model(data)
        y_reversed +=(output).cpu().detach().ravel().tolist()

    df_train["y_reversed"]=y_reversed
    t0=df_train[df_train["T"]==0]
    t0=t0.copy()
    t0.loc[:,"tau_hat"]=t0.loc[:,"y_reversed"]-t0.loc[:,"y_hat"]

    t1=df_train[df_train["T"]==1]
    t1=t1.copy()
    t1.loc[:,"tau_hat"]=t1.loc[:,"y_hat"]-t1.loc[:,"y_reversed"]

    ite_pred=np.concatenate([t1["tau_hat"],t0["tau_hat"]],axis=0)
    ite_true=np.concatenate([t1["tau"],t0["tau"]],axis=0)

    mse=MSE(ite_pred,ite_true)
    kl=kl_divergence(ite_pred,ite_true)
    ate_error=abs((np.mean(ite_pred)/np.mean(ite_true))-1)
    train_mse.append(mse)
    train_kl.append(kl)
    train_error.append(ate_error)

    print("Model %d has MSE on trainset:%f" % (i,np.mean(train_mse)),end='  ')
    print("Model %d has KL on trainset:%f" % (i,np.mean(train_kl)),end='  ')
    print("Model %d has ATE error on trainset:%f" % (i,np.mean(train_error)),end='  ')

    te_train["GAT ATE % Error",i]=np.mean(train_error)
    te_train["GAT MSE",i]=np.mean(train_mse)
    te_train["GAT KL Divergence",i]=np.mean(train_kl)

    y_predicted = []
    for data in test_loader:
        loss = 0
        data = data.to(device)
        output = model(data)
        label = data.y.to(device)
        y_predicted +=(output).cpu().detach().ravel().tolist()
    df_test["y_hat"]=y_predicted

    y_reversed=[]
    for data in reverse_test:
        data = data.to(device)
        output = model(data)
        y_reversed +=(output).cpu().detach().ravel().tolist()

    df_test["y_reversed"]=y_reversed
    t0=df_test[df_test["T"]==0]
    t0=t0.copy()
    t0.loc[:,"tau_hat"]=t0.loc[:,"y_reversed"]-t0.loc[:,"y_hat"]

    t1=df_test[df_test["T"]==1]
    t1=t1.copy()
    t1.loc[:,"tau_hat"]=t1.loc[:,"y_hat"]-t1.loc[:,"y_reversed"]

    ite_pred=np.concatenate([t1["tau_hat"],t0["tau_hat"]],axis=0)
    ite_true=np.concatenate([t1["tau"],t0["tau"]],axis=0)

    mse=MSE(ite_pred,ite_true)
    kl=kl_divergence(ite_pred,ite_true)
    ate_error=abs((np.mean(ite_pred)/np.mean(ite_true))-1)
    valid_mse.append(mse)
    valid_kl.append(kl)
    valid_error.append(ate_error)

    print("Model %d has MSE on validset:%f" % (i,np.mean(valid_mse)),end='  ')
    print("Model %d has KL on validset:%f" % (i,np.mean(valid_kl)),end='  ')
    print("Model %d has ATE error on validset:%f" % (i,np.mean(valid_error)),end='  ')

    te_valid["GAT ATE % Error",i]=np.mean(valid_error)
    te_valid["GAT MSE",i]=np.mean(valid_mse)
    te_valid["GAT KL Divergence",i]=np.mean(valid_kl)

R2:-0.424074  MSE:1.983721  KL:0.152656  MAE:1.128095  RMSE:1.408446  CVRMSE:0.930205  train_loss:
1.9837207841873168
R2:-0.065382  MSE:1.556780  KL:1.827174  MAE:0.991317  RMSE:1.247710  CVRMSE:0.823810  test_loss:
0.003115879774093628
R2:-0.050798  MSE:1.463751  KL:0.411650  MAE:0.966873  RMSE:1.209856  CVRMSE:0.799046  R2:0.044609  MSE:1.330850  KL:0.473220  MAE:0.918868  RMSE:1.153625  CVRMSE:0.761908  R2:0.093543  MSE:1.262686  KL:0.369659  MAE:0.892894  RMSE:1.123693  CVRMSE:0.742140  R2:0.121469  MSE:1.223785  KL:0.259094  MAE:0.880896  RMSE:1.106248  CVRMSE:0.730619  R2:0.135119  MSE:1.204770  KL:0.233147  MAE:0.873685  RMSE:1.097620  CVRMSE:0.724920  R2:0.141863  MSE:1.195376  KL:0.259911  MAE:0.868819  RMSE:1.093332  CVRMSE:0.722088  R2:0.160166  MSE:1.169881  KL:0.264850  MAE:0.860137  RMSE:1.081610  CVRMSE:0.714347  R2:0.165895  MSE:1.161900  KL:0.265663  MAE:0.854273  RMSE:1.077915  CVRMSE:0.711906  R2:0.172642  MSE:1.152502  KL:0.285137  MAE:0.854910  RMSE:1.073546  CVRMS

In [ ]:
te_train,te_valid

({('GAT ATE % Error', 0): 0.17259674217567733,
  ('GAT ATE % Error', 1): 0.21712214058461865,
  ('GAT ATE % Error', 2): 0.2130474967936029,
  ('GAT ATE % Error', 3): 0.19789900517021342,
  ('GAT ATE % Error', 4): 0.2098879209753104,
  ('GAT KL Divergence', 0): 0.08925940852484206,
  ('GAT KL Divergence', 1): 0.1373419849368817,
  ('GAT KL Divergence', 2): 0.11422556081455078,
  ('GAT KL Divergence', 3): 0.11350853119658112,
  ('GAT KL Divergence', 4): 0.1153807905117093,
  ('GAT MSE', 0): 0.08373246162299863,
  ('GAT MSE', 1): 0.08867946344822378,
  ('GAT MSE', 2): 0.0896931372530926,
  ('GAT MSE', 3): 0.08509088423387999,
  ('GAT MSE', 4): 0.09127337785933395,
  ('GCN ATE % Error', 0): 0.2442573234673373,
  ('GCN ATE % Error', 1): 0.2542773073706823,
  ('GCN ATE % Error', 2): 0.26949893023406335,
  ('GCN ATE % Error', 3): 0.28933613378115797,
  ('GCN ATE % Error', 4): 0.28993473798843694,
  ('GCN KL Divergence', 0): 0.19910949767671815,
  ('GCN KL Divergence', 1): 0.2270470242554361,


In [ ]:
gcn_error=[te_train[i] for i in te_train if 'GCN ATE % Error' in i[0]][0]
gcn_mse=[te_train[i] for i in te_train if 'GCN MSE' in i[0]][0]
gcn_kl=[te_train[i] for i in te_train if 'GCN KL Divergence' in i[0]][0]
gat_error=[te_train[i] for i in te_train if 'GAT ATE % Error' in i[0]][0]
gat_mse=[te_train[i] for i in te_train if 'GAT MSE' in i[0]][0]
gat_kl=[te_train[i] for i in te_train if 'GAT KL Divergence' in i[0]][0]
train_summary=pd.DataFrame(columns=[["Abs % Error of ATE","MSE","KL Divergence"]])
train_summary.loc["S learner(GCN)",:]=[gcn_error,gcn_mse,gcn_kl]
train_summary.loc["S learner(GAT)",:]=[gat_error,gat_mse,gat_kl]

gcn_error=[te_valid[i] for i in te_valid if 'GCN ATE % Error' in i[0]][0]
gcn_mse=[te_valid[i] for i in te_valid if 'GCN MSE' in i[0]][0]
gcn_kl=[te_valid[i] for i in te_valid if 'GCN KL Divergence' in i[0]][0]
gat_error=[te_valid[i] for i in te_valid if 'GAT ATE % Error' in i[0]][0]
gat_mse=[te_valid[i] for i in te_valid if 'GAT MSE' in i[0]][0]
gat_kl=[te_valid[i] for i in te_valid if 'GAT KL Divergence' in i[0]][0]
valid_summary=pd.DataFrame(columns=[["Abs % Error of ATE","MSE","KL Divergence"]])
valid_summary.loc["S learner(GCN)",:]=[gcn_error,gcn_mse,gcn_kl]
valid_summary.loc["S learner(GAT)",:]=[gat_error,gat_mse,gat_kl]

print("Validation Summary of Embedding(10d) & Unweighted")
valid_summary

Validation Summary of Embedding(10d) & Unweighted


,Abs % Error of ATE,MSE,KL Divergence
S learner(GCN),0.369335,0.077548,0.410573
S learner(GAT),0.259966,0.090975,0.153667


In [ ]:
train_summary.to_csv("Train_embedding(10d)_unweighted.csv")
valid_summary.to_csv("Valid_embedding(10d)_unweighted.csv")